# Convert from GeoTIFF to AiGIS format file (Single value)
- Input: ONC multi-band GeoTIFF map
- Output: Single value AiGIS format map

In [12]:
import numpy as np
import math
import rasterio
import pandas as pd
import geopandas as gpd
from shapely.geometry import Polygon, LineString, GeometryCollection
from shapely.ops import split
import csv

import datetime

In [13]:
def read_obj_file(file_path):
    # lists for vertices and faces
    vertices = []
    faces = []

    # open shape model OBJ file
    with open(file_path, 'r') as file:
        for line in file:
            # Vertex information starts from 'v'
            if line.startswith('v '):
                # Store xyz coordinate of vertices
                vertex = list(map(float, line.split()[1:])) 
                vertices.append(vertex)

            # Face information starts from 'f'
            elif line.startswith('f '):
                # Store the indeces which compose the face
                face = [int(idx.split('/')[0]) - 1 for idx in line.split()[1:]] 
                faces.append(face)

    return vertices, faces

def adjusted_coordinates(coords):
    longitudes = [lon for lon, lat in coords]
    difference  = max(longitudes) - min(longitudes)
    adjust_lon = (difference >= 300)
    adjusted_coords = [(lon - 360 if lon > 300 and adjust_lon else lon, lat) for lon, lat in coords]
    return adjusted_coords

In [14]:
# Input file: ONC multi-band mosaic map (GeoTIFF)
in_raster = './data/hyb2_onc_mos_20181003_MascotSeparation.tif'

# ONC filter wavelengths (micron)
# wl = [0.40,0.48,0.55,0.59,0.70,0.86,0.95]
band = 2 #set as 0.55 micron (v-band)
file_path = './data/SHAPE_SPC_49k_v20200323.obj' 

#output AiGIS format file paths
out_folder = './output/'
out_path =  out_folder + 'ONC_MSC_v-band_AiGIS.txt'

In [15]:
vertices, faces = read_obj_file(file_path)

#open raster
in_raster = './data/hyb2_onc_mos_20181003_MascotSeparation.tif'
raster = rasterio.open(in_raster)
data = raster.read()

ulx, uly = raster.xy(0, 0)
rows,cols = raster.shape
px = raster.transform[0]
py = raster.transform[4]*(-1)

ulx = ulx - px/2
uly = uly + py/2

lrx = ulx + cols*px
lry = uly - rows*py

print(ulx,uly,lrx,lry)

0.0 20.0734 360.03599999999994 -31.6226


In [16]:
vertices = vertices

root_xy = [math.sqrt(vertex[0]**2 + vertex[1]**2) for vertex in vertices]
root_xyz = [math.sqrt(vertex[0]**2 + vertex[1]**2 + vertex[2]**2) for vertex in vertices]

cos = [vertices[0] / r_xy if vertices[1] > 0 else vertices[0] / r_xy * (-1) for vertices, r_xy in zip(vertices, root_xy)]
sin = [vertices[2] / r_xyz for vertices, r_xyz in zip(vertices, root_xyz)]

lon = [(math.acos(c) * 180 / math.pi) if vertices[1] > 0 else (180 + math.acos(c) * 180 / math.pi)
    for vertices, c in zip(vertices, cos)]
lon_np = np.array(lon)

lat = [(math.asin(s) * 180 / math.pi) for s in sin]
lat_np = np.array(lat)

face_coordinates = []
count_result = []

for face in faces:
    coordinates = []
    for idx in face:
        lon_val = lon[idx]
        lat_val = lat[idx]
        
        coordinates.append((lon_val,lat_val)) 
    
    # If the longitude exceeds 360, adjust the coordinates
    face_coordinates.append(adjusted_coordinates(coordinates))
    count_result.append(1)

In [17]:
# convert map coordicates to image coordinates
ras_v = []
face_coordinates_np = np.array(face_coordinates, dtype=object)
x_coords_np = np.array([[coord[0] for coord in coords] for coords in face_coordinates_np])
y_coords_np = np.array([[coord[1] for coord in coords] for coords in face_coordinates_np])

# mean map coordiantes (gx, gy)
gx = np.mean(x_coords_np, axis=1)
gx = np.where(gx < 0, gx + 360, gx)
gy = np.mean(y_coords_np, axis=1)

# calculate image coordinates
img_x = np.floor(gx / px).astype(int)
img_y = np.floor((uly - gy) / py).astype(int)

# if the length of x_coords_np and y_coords_np is not 3, set the image coordinates to 0
valid_indices = (x_coords_np.shape[1] == 3) & (y_coords_np.shape[1] == 3) & (img_x >= 0) & (img_y >= 0) & (img_x < raster.shape[1]) & (img_y < raster.shape[0])

print('Number of valid indices: ', np.sum(valid_indices))

invalid_indices = ~valid_indices
img_x[invalid_indices] = 0
img_y[invalid_indices] = 0

num_bands = raster.count # band count for the input raster
ras_v = [] # empty list to store pixel values

band_data = raster.read(band)
values = band_data[img_y, img_x]
ras_v.append(values)

ras_v = np.array(ras_v).T

# for invalid indices, set the pixel values to 0
ras_v[invalid_indices] = 0.0
count_result = np.zeros_like(valid_indices, dtype=int)
count_result[valid_indices] = 1

valid_indices2 = (ras_v != -1.0) 
ras_v[~valid_indices2] = 0.0
count_result[~valid_indices2[:,0]] = 0

Number of valid indices:  20865


In [ ]:
#Output AiGIS format file
with open(out_path, 'w') as f:
    
    # Set the header information
    f.write("ONC_reflectance\n")
    f.write("-\n")
    f.write(f"{len(count_result)}\n")

    # write data (row 1: polygon number, row 2: pixel values)
    for idx, row in enumerate(ras_v, start=1):
        line = f"{idx}\t" + '\t'.join([f"{value:.6f}" for value in row]) + '\n'
        f.write(line)


print('complete!')

complete!
